In [ ]:
# Colab setup

# try:
#     from dlroms import *
# except:
#     !pip install git+https://github.com/NicolaRFranco/dlroms.git
#     from dlroms import *

# TODO: install dlroms_bayesian

In [2]:
import torch
import numpy as np
import os
import matplotlib.pyplot as plt

from dolfin import *
from dlroms import *
from dlroms.dnns import *

from dlroms_bayesian.bayesian import Bayesian
from dlroms_bayesian.svgd import SVGD
from dlroms_bayesian.utils import *

from IPython.display import clear_output as clc

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
set_seeds(0)

In [3]:
# Domain and mesh definition

domain = fe.rectangle((0.0, 0.0), (1.0, 1.0))
mesh = fe.mesh(domain, stepsize=0.05)
V = fe.space(mesh, 'CG', 1) # 441 dofs
l2 = L2(V) # L2 norm

if torch.cuda.is_available():
	l2.cuda()

clc()

In [4]:
# Load train and test data

path_train = os.path.join(os.getcwd(), "snapshots", "snapshots_train_C.npz")
data_train = np.load(path_train)

N_train = data_train['K'].shape[0]
K_train = torch.tensor(data_train['K'].astype(np.float32)).to(device)
p_train = torch.tensor(data_train['p'].astype(np.float32)).to(device)
u_x_train = torch.tensor(data_train['u_x'].astype(np.float32)).to(device)
u_y_train = torch.tensor(data_train['u_y'].astype(np.float32)).to(device)

path_test = os.path.join(os.getcwd(), "snapshots", "snapshots_test_C.npz")
data_test = np.load(path_test)

N_test = data_test['K'].shape[0]
K_test = torch.tensor(data_test['K'].astype(np.float32)).to(device)
p_test = torch.tensor(data_test['p'].astype(np.float32)).to(device)
u_x_test = torch.tensor(data_test['u_x'].astype(np.float32)).to(device)
u_y_test = torch.tensor(data_test['u_y'].astype(np.float32)).to(device)

In [ ]:
# Model definition

m = 16

# Encoder
psi = Reshape(1, 21, 21) + \
      Conv2D(6, (1, m), stride=1) + \
      Conv2D(7, (m, 2 * m), stride=1) + \
      Conv2D(7, (2 * m, 4 * m), stride=1, activation=None)

# Decoder
psi_prime = Deconv2D(7, (4 * m, 2 * m), stride=1) + \
            Deconv2D(7, (2 * m, m), stride=1) + \
            Deconv2D(6, (m, 1), stride=1, activation=None) + \
            Reshape(-1)

print("Encoder trainable parameters:", psi.dof())
print("Decoder trainable parameters:", psi_prime.dof())

# Pressure Bayesian network

In [15]:
# Bayesian network for pressure

p_model = DFNN(psi, psi_prime)

p_bayes = Bayesian(p_model)

if torch.cuda.is_available():
	p_bayes.cuda()

N_particles = 10

p_trainer = SVGD(p_bayes, n_samples=N_particles)
p_trainer.He()
p_bayes.set_trainer(p_trainer)

# p_bayes.train(K_train, p_train, ntrain=N_train, lr=1e-3, lr_noise=1e-3, loss=mse(l2), epochs=5000)
history = p_bayes.train(K_train, p_train, ntrain=N_train, lr=1e-3, lr_noise=1e-3, loss=mse(l2), epochs=5000, track_history=True)

In [10]:
# Compute mean and variance of predictions

p_pred_bayes_mean_train, p_pred_bayes_var_train = p_bayes.sample(K_train, n_samples=N_particles)
p_pred_bayes_mean, p_pred_bayes_var = p_bayes.sample(K_test, n_samples=N_particles)

In [ ]:
# Compute relative error

error_train = mre(l2)(p_train, p_pred_bayes_mean_train)
error_test = mre(l2)(p_test, p_pred_bayes_mean)

print(f"Relative training error: {100 * torch.mean(error_train):.2f}")
print(f"Relative test error: {100 * torch.mean(error_test):.2f}")

In [ ]:
# Plot a random snapshot

idx = 50

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
vmin, vmax = torch.min(p_test), torch.max(p_test)
fe.plot(p_test[idx], V, cmap='jet', vmin=vmin, vmax=vmax, colorbar=True)
plt.title("True pressure")
plt.subplot(1, 3, 2)
fe.plot(p_pred_bayes_mean[idx], V, cmap='jet', vmin=vmin, vmax=vmax, colorbar=True)
plt.title("Predicted mean pressure")
plt.subplot(1, 3, 3)
fe.plot(p_pred_bayes_var[idx], V, cmap='jet', vmin=vmin, vmax=vmax, colorbar=True)
plt.title("Pressure variance")
plt.show()

In [ ]:
# Save trainer state

p_trainer.save_particles(os.path.join('checkpoints', 'p_particles.pth'))

In [ ]:
# Save training history

import pickle

with open('history.pkl', 'wb') as f:
    pickle.dump(history, f)

# Velocity Bayesian network (x-component)

In [ ]:
# Bayesian network for velocity (x-component)

u_x_model = DFNN(psi, psi_prime)

u_x_bayes = Bayesian(u_x_model)

if torch.cuda.is_available():
	u_x_bayes.cuda()

u_x_trainer = SVGD(u_x_bayes, n_samples=N_particles)
u_x_trainer.He()
u_x_bayes.set_trainer(u_x_trainer)

u_x_bayes.train(K_train, u_x_train, ntrain=N_train, lr=1e-3, lr_noise=1e-3, loss=mse(l2), epochs=5000)

In [ ]:
# Compute mean and variance of predictions

u_x_pred_bayes_mean_train, u_x_pred_bayes_var_train = u_x_bayes.sample(K_train, n_samples=N_particles)
u_x_pred_bayes_mean, u_x_pred_bayes_var = u_x_bayes.sample(K_test, n_samples=N_particles)

In [ ]:
# Compute relative error

error_train = mre(l2)(u_x_train, u_x_pred_bayes_mean_train)
error_test = mre(l2)(u_x_test, u_x_pred_bayes_mean)

print(f"Relative training error: {100 * torch.mean(error_train):.2f}")
print(f"Relative test error: {100 * torch.mean(error_test):.2f}")

In [ ]:
# Plot a random snapshot

idx = 50

plt.figure(figsize=(18, 6))
vmin, vmax = torch.min(u_x_test), torch.max(u_x_test)
plt.subplot(1, 3, 1)
fe.plot(u_x_test[idx], V, cmap='jet', vmin=vmin, vmax=vmax, colorbar=True)
plt.title("True velocity (x-comp.)")
plt.subplot(1, 3, 2)
fe.plot(u_x_pred_bayes_mean[idx], V, cmap='jet', vmin=vmin, vmax=vmax, colorbar=True)
plt.title("Predicted mean velocity (x-comp.)")
plt.subplot(1, 3, 3)
fe.plot(u_x_pred_bayes_var[idx], V, cmap='jet', vmin=vmin, vmax=vmax, colorbar=True)
plt.title("Velocity variance (x-comp.)")
plt.show()

In [ ]:
# Save trainer state

u_x_trainer.save_particles(os.path.join('checkpoints', 'u_x_particles.pth'))

# Velocity Bayesian network (y-component)

In [ ]:
# Bayesian network for velocity (y-component)

u_y_model = DFNN(psi, psi_prime)

u_y_bayes = Bayesian(u_y_model)

if torch.cuda.is_available():
	u_y_bayes.cuda()

u_y_trainer = SVGD(u_y_bayes, n_samples=N_particles)
u_y_trainer.He()
u_y_bayes.set_trainer(u_y_trainer)

u_y_bayes.train(K_train, u_y_train, ntrain=int(0.9*N_train), lr=1e-3, lr_noise=1e-3, loss=mse(l2), epochs=5000)

In [ ]:
# Compute mean and variance of predictions

u_y_pred_bayes_mean_train, u_y_pred_bayes_var_train = u_y_bayes.sample(K_train, n_samples=N_particles)
u_y_pred_bayes_mean, u_y_pred_bayes_var = u_y_bayes.sample(K_test, n_samples=N_particles)

In [ ]:
# Compute relative error

error_train = mre(l2)(u_y_train, u_y_pred_bayes_mean_train)
error_test = mre(l2)(u_y_test, u_y_pred_bayes_mean)

print(f"Relative training error: {100 * torch.mean(error_train):.2f}")
print(f"Relative test error: {100 * torch.mean(error_test):.2f}")

In [ ]:
# Plot a random snapshot

idx = 50

plt.figure(figsize=(18, 6))
vmin, vmay = torch.min(u_y_test), torch.may(u_y_test)
plt.subplot(1, 3, 1)
fe.plot(u_y_test[idx], V, cmap='jet', vmin=vmin, vmay=vmay, colorbar=True)
plt.title("True velocity (y-comp.)")
plt.subplot(1, 3, 2)
fe.plot(u_y_pred_bayes_mean[idx], V, cmap='jet', vmin=vmin, vmay=vmay, colorbar=True)
plt.title("Predicted mean velocity (y-comp.)")
plt.subplot(1, 3, 3)
fe.plot(u_y_pred_bayes_var[idx], V, cmap='jet', vmin=vmin, vmay=vmay, colorbar=True)
plt.title("Velocity variance (y-comp.)")
plt.show()

In [ ]:
# Save trainer state

u_y_trainer.save_particles(os.path.join('checkpoints', 'u_y_particles.pth'))